# Turbo 
> Iteration manager

In [1]:
import pandas as pd
import numpy as np

In [2]:
import math
class turboBase(object):
    def __init__(self,length, batch_size=16,shuffle = True):
        self.length = length
        self.shuffle = shuffle 
        self.batch_size = batch_size
        self.march_funcs = {}
        self.march_order = []
        self.shuffle_data()
        self.idx = 0
        
    def epoch(self):
        gen = iter(self)
        t= tqdm(range(len(self)))
        for i in t:
            rt = next(gen)
            if rt["success"]==False:
                break
                
    def __repr__(self):
        return f"<turbo bs:{self.batch_size} len:{len(self)}>"
        
    def shuffle_data(self):
        if self.shuffle:
            self.order = np.random.permutation(len(self))
        
    def __len__(self):
        return math.ceil(float(self.length)/float(self.batch_size))
    
    def __iter__(self):
        raise NotImplemented("Please define the __iter__ function first")
    
    def march(self,f):
        funame = f.__name__
        self.march_funcs[funame] = f
        if funame not in self.march_order:
            self.march_order.append(funame)
        def wraper(*args,**kwargs):
            return f(*args,**kwargs)
        return wraper
    
    def __call__(self,x):
        for m in self.march_order:
            x = self.march_funcs[m](x)
        return x

In [3]:
df = pd.read_csv("data/netflix_titles.csv")

In [4]:
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
...,...,...,...,...,...,...,...,...,...,...,...,...
6229,80000063,TV Show,Red vs. Blue,NaN,"Burnie Burns, Jason Saldaña, Gustavo Sorola, G...",United States,NaN,2015,NR,13 Seasons,"TV Action & Adventure, TV Comedies, TV Sci-Fi ...","This parody of first-person shooter games, mil..."
6230,70286564,TV Show,Maron,NaN,"Marc Maron, Judd Hirsch, Josh Brener, Nora Zeh...",United States,NaN,2016,TV-MA,4 Seasons,TV Comedies,"Marc Maron stars as Marc Maron, who interviews..."
6231,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,NaN,NaN,NaN,NaN,2016,NaN,60 min,Movies,Nursery rhymes and original music for children...
6232,70281022,TV Show,A Young Doctor's Notebook and Other Stories,NaN,"Daniel Radcliffe, Jon Hamm, Adam Godley, Chris...",United Kingdom,NaN,2013,TV-MA,2 Seasons,"British TV Shows, TV Comedies, TV Dramas","Set during the Russian Revolution, this comic ..."


In [5]:
from tqdm import tqdm
class dfTurbo(turboBase):
    def __init__(self,df, batch_size=16, shuffle=False):
        super().__init__(len(df),batch_size=batch_size,shuffle=shuffle)
        self.df = df
        
    def __iter__(self):
        self.idx = 0
        return self
        
    def __next__(self):
        if self.shuffle:
            a = self.order[self.idx]
        else:
            a = self.idx
        b=a+1
        try:
            self.idx+=1
            dt = self(self.df[self.batch_size*a:self.batch_size*b])
            return  {"data":dt,"success":True}
        except Exception as e:
            self.idx=0
            return {"data":{"msg":str(e)}, "success":False}

In [6]:
dft = dfTurbo(df)

In [8]:
@dft.march
def directors(df):
    print(list(df[~df.director.isna()]["director"]))
    return df

In [9]:
dft.epoch()

 51%|█████     | 199/390 [00:00<00:00, 901.99it/s]

['Richard Finn, Tim Maltby', 'Fernando Lebrija', 'Gabe Ibáñez', 'Rodrigo Toro, Francisco Schultz', 'Henrik Ruben Genz', 'José Miguel Contreras', 'Daniel Alfredson', 'Munjal Shroff, Tilak Shetty', 'Munjal Shroff, Tilak Shetty', 'Tilak Shetty']
['Tilak Shetty', 'Munjal Shroff, Tilak Shetty', 'Gaspar Noé', "Tom O'Brien", 'Antoine Bardou-Jacquet', 'Mitch Dickman', 'Brad Anderson', 'Austin Stark', 'Hannah Fidell', 'Madeleine Gavin', 'Sopon Sukdapisit', 'Kevin R. Adams, Joe Ksander', 'Ian Samuels']
['Franck Ribière', 'Daniele Thompson', 'Anubhav Sinha', 'Maha Venkatesh', 'Tharun Bhascker', 'Şenol Sönmez', 'Tom Gianas, Ross R. Shuman', 'Rajkumar Hirani', 'Robert Osman, Nathanael Wiseman', 'Jon Spira', 'Mez Tharatorn', 'Adisorn Tresirikasem', 'Mez Tharatorn']
['Banjong Pisanthanakun', 'Banjong Pisanthanakun, Paween Purikitpanya, Songyos Sugmakanan, Parkpoom Wongpoom, Visute Poolvoralaks', 'Banjong Pisanthanakun, Parkpoom Wongpoom', 'Chayanop Boonprakob', 'Susan Johnson', 'Lynn Shelton', 'Farha

100%|██████████| 390/390 [00:00<00:00, 1009.41it/s]


['Mario Rouleau', 'Santiago Diaz, Pablo Martin Farina', 'Scott Zabielski', 'Raúl Campos, Jan Suter', 'Ahmed Khaled Moussa', 'Néstor Sánchez Sotelo', 'Aniruddha Roy Chowdhury', 'Chester Tam', 'Mohit Suri']
['Terry Gilliam, Terry Jones', 'Jon Schnitzer', 'Craig Elrod', 'Benjamin Cohen, Nicolas Cotto', 'Mike Rossiter', 'Michael McKay', 'Charles Stone III', 'Matt Angel, Suzanne Coote', 'Chris McCoy', 'Sofia Coppola', 'Chris Smith', 'Jonathan Helpert']
["André D'Elia", 'Matías Gueilburt', 'Ivan Ayr', 'Chris Baugh', 'Jenny Carchman', 'Tyler Perry', 'Raúl Campos, Jan Suter', 'Kobun Shizuno, Hiroyuki Seshita', 'Neal Brennan', 'G.J. Echternkamp']
['Jehane Noujaim', 'Caryn Waechter', 'Nicholas Stoller', 'Tim Hill', 'Numa Perrier', 'Brian Helgeland', 'Yoshiyuki Momose', 'Danny Boyle', 'Colin Strause, Greg Strause', 'Ethan Coen, Joel Coen', 'Jean-Marc Vallée', 'Marc Abraham', 'Steve Barker']
['Shawn Levy', 'Rima Das', 'Paco Plaza', 'Andrew Goth', 'Jeffrey G. Hunt', 'William Kaufman', 'Marek Kanie